## Extração de Dados da API Open Data SUS

O objetivo desse notebook será gerar um CSV com um conjunto de dados associados de forma a possibilitar diferentes análises que faremos nas etapas seguintes. 

A idéia é ter em cada linha da tabela a informação do município e como ele estava no dia em questão e dessa maneira poder ser analisado evolução de casos e situações ao decorrer do tempo. 

Metas: 
- [x] Integrar dados agrupados por município com informações descritivas e geográficas 
- [x] Obter indicadores de casos separados por cada dia 
- [x] Obter informação do aumento do número suspeitos, confirmados e óbitos por dia
- [x] Salvar dados em CSV
- [ ] Salvar dados em Excel (XLSX) 

### Links Referência OpenSUS

1. [Conjunto de APIs](https://github.com/EscolaDeSaudePublica/coronavirusAPI/issues/17)

### Links Interessantes
Abaixo alguns links utilizados com maiores instruções e detalhamentos sobre os procedimentos utilizados. 

1. [Convertendo JSON to dataframe](https://stackoverflow.com/questions/42518864/convert-json-data-from-request-into-pandas-dataframe)

2. [Básico de Plotly](https://paulovasconcellos.com.br/como-criar-gr%C3%A1ficos-interativos-utilizando-plotly-e-python-3eb6eda57a2b)

3. [Como Trabalhar com Datas](https://dicasdepython.com.br/python-como-converter-date-em-string-com-formatacao/)


### Instalando dependências
As dependências utilizadas estão a seguir. Basta descomentar e executar a célula abaixo para que possa ser instalada. 

In [1]:
# %conda install -c anaconda requests
# %conda install -c plotly chart-studio
# %conda install -c plotly plotly

### Iniciando dependências e configurações básicas

In [2]:
import pandas as pd
import requests
from datetime import date,datetime, timedelta
import numpy as np

### Criando um método auxiliar para consumir as APIs 

Normalmente todas APIs seguem o padrão de alguns campos como idMunicipio (ID IBGE do município) e data (dia no formato YYYY-MM-DD). 

In [3]:
def get_dataframe(api_url, data='', id_municipio=''):
    '''Método básico para converter uma chamada a API em um objeto Dataframe'''
    
    api_url = api_url + '?'
    if(id_municipio != ''): 
        api_url = '{}idMunicipio={}&'.format(api_url, id_municipio)
    
    if(data != ''): 
        api_url = '{}data={}&'.format(api_url, data)
    
    result = requests.get(api_url)
    result = result.json()    
    result = pd.DataFrame.from_dict(result)
    
    if(id_municipio != ''): 
        result.insert(0, 'idMunicipio', id_municipio)
        
    return result

In [4]:
municipios          = get_dataframe('https://indicadores.integrasus.saude.ce.gov.br/api/municipio')
macro_regiao        = get_dataframe('https://indicadores.integrasus.saude.ce.gov.br/api/macro-regiao')
regiao_saude        = get_dataframe('https://indicadores.integrasus.saude.ce.gov.br/api/regiao-saude')
qtd_por_municipios  = get_dataframe('https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-por-municipio')

In [5]:
qtd_por_municipios.head(3)

,tipo,idMunicipio,qtdSuspeito,municipio,qtdObito,qtdConfirmado
0,Em investigação,230010.0,4.0,ABAIARA,NaN,NaN
1,Em investigação,230015.0,5.0,ACARAPE,NaN,NaN
2,Em investigação,230020.0,28.0,ACARAU,NaN,NaN


In [6]:
municipios.head(3)

,id,idMacroRegiao,idRegiaoSaude,descricao,latitude,longitude
0,230010,8,19,ABAIARA,-735542,-3904079
1,230015,1,2,ACARAPE,-422124,-3870508
2,230020,2,12,ACARAU,-288469,-4011870


In [7]:
macro_regiao.head(3)

,id,descricao
0,8,CARIRI
1,1,FORTALEZA
2,10,LITORAL LESTE/JAGUARIBE


### Unir Município e Macro Região
Como unir os dados dessas tabelas? Vamos usar a função [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) do pandas, o qual é semelhante ao PROCV do Excel.  

In [8]:
# Vamos unir a tabela de municípios com as suas respectivas macro regiões. 
municipio_macro_regiao = municipios.merge(macro_regiao, 
                               left_on='idMacroRegiao', 
                               right_on='id', 
                               suffixes=['Municipio','MacroRegiao'])

municipio_macro_regiao.head(3)

,idMunicipio,idMacroRegiao,idRegiaoSaude,descricaoMunicipio,latitude,longitude,idMacroRegiao,descricaoMacroRegiao
0,230010,8,19,ABAIARA,-735542,-3904079,8,CARIRI
1,230030,8,18,ACOPIARA,-609062,-3945153,8,CARIRI
2,230060,8,20,ALTANEIRA,-700102,-3973774,8,CARIRI


### Unir Região de Saúde

In [9]:
regiao_saude.head(3)

,id,idMacroRegiao,descricao
0,1,1,1ª REGIÃO FORTALEZA
1,2,1,3ª REGIÃO MARACANAÚ
2,3,1,2ª REGIÃO CAUCAIA


In [10]:
# Vamos unir a tabela de municípios com as suas respectivas macro regiões. 
municipio_macro_regiao = municipio_macro_regiao.merge(regiao_saude, 
                               left_on='idRegiaoSaude', 
                               right_on='id', suffixes=['','RegiaoSaude'])

municipio_macro_regiao.head(3)

,idMunicipio,idMacroRegiao,idRegiaoSaude,descricaoMunicipio,latitude,longitude,idMacroRegiao,descricaoMacroRegiao,id,idMacroRegiaoRegiaoSaude,descricao
0,230010,8,19,ABAIARA,-735542,-3904079,8,CARIRI,19,8,19ª REGIÃO BREJO SANTO
1,230170,8,19,AURORA,-693893,-3896717,8,CARIRI,19,8,19ª REGIÃO BREJO SANTO
2,230200,8,19,BARRO,-717277,-3877582,8,CARIRI,19,8,19ª REGIÃO BREJO SANTO


In [11]:
# apaga algumas colunas desnecessárias
municipio_macro_regiao.drop(columns=['idMacroRegiaoRegiaoSaude','id'], index=1, inplace=True)

# algumas colunas estão duplicadas e podem nos atrapalhar. Removendo. 


municipio_macro_regiao = municipio_macro_regiao.loc[:,~ municipio_macro_regiao.columns.duplicated()]
municipio_macro_regiao.head(3)

,idMunicipio,idMacroRegiao,idRegiaoSaude,descricaoMunicipio,latitude,longitude,descricaoMacroRegiao,descricao
0,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO
2,230200,8,19,BARRO,-717277,-3877582,CARIRI,19ª REGIÃO BREJO SANTO
3,230250,8,19,BREJO SANTO,-748943,-3898571,CARIRI,19ª REGIÃO BREJO SANTO


### Ajustar nomes de colunas dos municípios e regiões

In [12]:
municipio_macro_regiao.rename(columns={"descricao": "regiaoSaude", 
                                       "descricaoMunicipio": "municipio",
                                       "descricaoMacroRegiao": "macroRegiao",
                                       "latitude":"lat", 
                                       "longitude":"lon"},
                             inplace=True)

In [13]:
municipio_macro_regiao.shape 
# Quantos municipios tem o Ceará? TUDO OK
# https://www.google.com/search?q=quantos+municipios+tem+o+ceara&oq=quantos+municipios+tem+o+ceara&aqs=chrome..69i57j0l2.5808j0j7&sourceid=chrome&ie=UTF-8

(183, 8)

### Unir dados à Quantidade Por Município
- Agora vamos unir os dados de municipio/macro-região aos dados de quantidades por municipio

In [14]:
qtd_por_municipios.head(3)

,tipo,idMunicipio,qtdSuspeito,municipio,qtdObito,qtdConfirmado
0,Em investigação,230010.0,4.0,ABAIARA,NaN,NaN
1,Em investigação,230015.0,5.0,ACARAPE,NaN,NaN
2,Em investigação,230020.0,28.0,ACARAU,NaN,NaN


In [15]:
qtd_por_municipios.shape

(258, 6)

In [16]:
qtd_por_municipios[qtd_por_municipios.idMunicipio.isna()]

,tipo,idMunicipio,qtdSuspeito,municipio,qtdObito,qtdConfirmado
175,Em investigação,NaN,2301.0,Sem informação,NaN,NaN
257,Confirmado,NaN,NaN,Sem informação,NaN,41.0


### Pergunta
[Pergunta](https://github.com/EscolaDeSaudePublica/coronavirusAPI/issues/17#issuecomment-612133737) 
 @victorMagalhaesPacheco, na API qtd-por-municipio em alguns não há informação do município. Pode explicar melhor quando isso ocorre? É realmente esperado?


In [17]:
qtd_por_municipios = qtd_por_municipios[~qtd_por_municipios['idMunicipio'].isna()]
qtd_por_municipios['idMunicipio'] = qtd_por_municipios['idMunicipio'].astype(int)
qtd_por_municipios.shape

(256, 6)

In [18]:
fortaleza = qtd_por_municipios[qtd_por_municipios.municipio == 'FORTALEZA']
display(fortaleza)

,tipo,idMunicipio,qtdSuspeito,municipio,qtdObito,qtdConfirmado
54,Em investigação,230440,6388.0,FORTALEZA,NaN,NaN
182,Óbito,230440,NaN,FORTALEZA,86.0,NaN
211,Confirmado,230440,NaN,FORTALEZA,NaN,1846.0


In [19]:
# Como fica apenas FORTALEZA agrupado por id do município
qtd_por_municipios = qtd_por_municipios.groupby('idMunicipio').sum()
qtd_por_municipios.head(3)


,qtdSuspeito,qtdObito,qtdConfirmado
idMunicipio,,,
230010,4.0,0.0,0.0
230015,5.0,0.0,0.0
230020,28.0,0.0,0.0


In [20]:
qtd_por_municipios.reset_index(inplace=True)
display(qtd_por_municipios.head(3))

,idMunicipio,qtdSuspeito,qtdObito,qtdConfirmado
0,230010,4.0,0.0,0.0
1,230015,5.0,0.0,0.0
2,230020,28.0,0.0,0.0


In [21]:
qtd_por_municipios[qtd_por_municipios.idMunicipio == 230440]

,idMunicipio,qtdSuspeito,qtdObito,qtdConfirmado
54,230440,6388.0,86.0,1846.0


In [22]:
municipio_macro_regiao.head(1)

,idMunicipio,idMacroRegiao,idRegiaoSaude,municipio,lat,lon,macroRegiao,regiaoSaude
0,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO


In [23]:
df_parcial  = municipio_macro_regiao.merge(qtd_por_municipios, on='idMunicipio')
df_parcial.head(3)

,idMunicipio,idMacroRegiao,idRegiaoSaude,municipio,lat,lon,macroRegiao,regiaoSaude,qtdSuspeito,qtdObito,qtdConfirmado
0,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO,4.0,0.0,0.0
1,230200,8,19,BARRO,-717277,-3877582,CARIRI,19ª REGIÃO BREJO SANTO,5.0,0.0,0.0
2,230250,8,19,BREJO SANTO,-748943,-3898571,CARIRI,19ª REGIÃO BREJO SANTO,6.0,0.0,0.0


In [24]:
df_parcial.shape

(174, 11)

In [25]:
# Padronizar as latitudes / longitudes. 
# atualmente os dados estão em formato string (texto) e o valor não é um decimal válido. 

# Convertendo de texto (str) para numérico (int)
df_parcial['lat'] = df_parcial['lat'].astype(int)
df_parcial['lon'] = df_parcial['lon'].astype(int)

# utilizando formato mais comum de sistemas geográficos (decimal): dividindo por 10000
df_parcial['lat'] = df_parcial['lat'] /100000
df_parcial['lon'] = df_parcial['lon'] /100000

In [26]:
df_parcial.head(3)

,idMunicipio,idMacroRegiao,idRegiaoSaude,municipio,lat,lon,macroRegiao,regiaoSaude,qtdSuspeito,qtdObito,qtdConfirmado
0,230010,8,19,ABAIARA,-7.35542,-39.04079,CARIRI,19ª REGIÃO BREJO SANTO,4.0,0.0,0.0
1,230200,8,19,BARRO,-7.17277,-38.77582,CARIRI,19ª REGIÃO BREJO SANTO,5.0,0.0,0.0
2,230250,8,19,BREJO SANTO,-7.48943,-38.98571,CARIRI,19ª REGIÃO BREJO SANTO,6.0,0.0,0.0


#### Como estão apresentados os dados de Fortaleza? 

In [27]:
df_parcial[df_parcial.municipio == 'FORTALEZA']

,idMunicipio,idMacroRegiao,idRegiaoSaude,municipio,lat,lon,macroRegiao,regiaoSaude,qtdSuspeito,qtdObito,qtdConfirmado
67,230440,1,1,FORTALEZA,-3.71701,-38.53926,FORTALEZA,1ª REGIÃO FORTALEZA,6388.0,86.0,1846.0


### Informações Por Sexo e Faixa Etária

In [28]:
## TODO
# /api/coronavirus/qtd-por-faixa-etaria-sexo?data=2020-04-09&tipo=Confirmado&idMunicipio=

### Obter Contagens Por Dia

Liberaram uma API muito boa que dá o resultado por dia, mas um problema é que não temos a identificação do município no retorno. 
Contudo, como posso passar o município como parâmetro para a API, irei utilizado para iterar na lista de municípios, realizar uma consulta específica por município e unir os dados. 
Observer que ficou bem mais fácil que o código complexo acima que está comentado. 

In [29]:
api_endpoint = "https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-por-dia-tipo"

qtd_por_dia_municipio = [get_dataframe(api_endpoint, id_municipio=idm) for idm in municipios.id]


In [30]:
api_endpoint = "https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-por-dia-tipo"

qtd_por_dia = pd.concat(qtd_por_dia_municipio, axis=0, ignore_index=True, sort=True)

qtd_por_dia['data'] = pd.to_datetime(qtd_por_dia['data'], format='%d/%m/%Y')
qtd_por_dia.head(3)
qtd_por_dia.sort_values(['data','idMunicipio'], inplace=True)

In [31]:
display("Menor data {} e maior data {}".format(qtd_por_dia.data.min(), qtd_por_dia.data.max()))
display("Quantidade de municípios com código vazio: {}".format(qtd_por_dia[qtd_por_dia.idMunicipio.isna()].shape[0]))
display(qtd_por_dia.head(5))

'Menor data 2020-01-08 00:00:00 e maior data 2020-09-11 00:00:00'

'Quantidade de municípios com código vazio: 0'

,data,idMunicipio,quantidade,tipo
914,2020-01-08,230765,1.0,Em investigação
871,2020-01-14,230740,1.0,Em investigação
1114,2020-01-26,230970,1.0,Em investigação
467,2020-01-27,230440,1.0,Em investigação
385,2020-02-07,230420,1.0,Em investigação


### Inconsistência: Dados de 2019 e no futuro 
Inconsistência na informação: dados de Março de 2019 e Agosto de 2020.  
Reportado em: https://github.com/EscolaDeSaudePublica/coronavirusAPI/issues/17#issuecomment-612434411

### Limpeza de dados inconsistentes

In [32]:
display(qtd_por_dia.shape)
inicio = datetime(year=2020,month=2, day=1)
fim    = datetime.today()
qtd_por_dia = qtd_por_dia[(qtd_por_dia['data'] > inicio) & (qtd_por_dia['data'] <= fim)]

display(qtd_por_dia.shape)

(1493, 4)

(1486, 4)

In [33]:
# fig = px.bar(qtd_por_dia, x="data", y="quantidade", color='tipo', barmode='group')
# fig.show()

In [34]:
# qtd_por_dia['log_quantidade'] = np.log(qtd_por_dia.quantidade)
# fig = px.bar(qtd_por_dia, x="data", y="log_quantidade", color='tipo', barmode='group')
# fig.show()

In [35]:
qtd_por_dia = municipio_macro_regiao.merge(qtd_por_dia, on='idMunicipio')

In [36]:
display(qtd_por_dia.shape)
display(qtd_por_dia.head(3))

(1481, 11)

,idMunicipio,idMacroRegiao,idRegiaoSaude,municipio,lat,lon,macroRegiao,regiaoSaude,data,quantidade,tipo
0,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO,2020-03-20,1.0,Em investigação
1,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO,2020-03-29,1.0,Em investigação
2,230010,8,19,ABAIARA,-735542,-3904079,CARIRI,19ª REGIÃO BREJO SANTO,2020-04-02,1.0,Em investigação


### Número de Exames Por Dia

In [37]:
### Cria dataframe com todos os dias desde o mínimo registro até hoje
df_dia = qtd_por_dia
start = df_dia.data.min()
end   = df_dia.data.max()
delta = end - start
add_days = lambda i: (start + timedelta(days=i))#.strftime('%Y-%m-%d')

all_times = [[1, add_days(i)] for i in range(delta.days + 1) ]
all_times = pd.DataFrame(all_times, columns = ['id','data']) 
all_times.head(3)

,id,data
0,1,2020-02-07
1,1,2020-02-08
2,1,2020-02-09


In [38]:
# api_endpoint = "https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-exames"

# result = [get_dataframe(api_endpoint, 
#                   id_municipio=idm, 
#                   data=data) 
#         for idm in municipios.id 
#             for data in all_times.data.dt.strftime('%Y-%m-%d')
#    ]
# result

In [39]:
# result = pd.concat(result)

In [40]:
# datas = [data for idm in municipios.id for data in all_times.data.dt.strftime('%Y-%m-%d')]

In [41]:
# result['data'] = datas

In [42]:
# result.groupby('idMunicipio').max()

### Criar coluna com dados cumulativos 

In [43]:
municipio_data = qtd_por_dia
# qtd_por_dia['exames'] = qtd_por_dia.apply()
get_dataframe("https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-exames", 
              data='2020-04-07', 
              id_municipio=230440)

,idMunicipio,tipo,quantidadeExame
0,230440,QtdExame,7302


In [44]:
all_mun = df_dia.loc[:,['municipio','macroRegiao','regiaoSaude']].drop_duplicates()
all_mun['id'] = 1
all_times = all_times.merge(all_mun, how='outer', on='id')
all_times.drop(['id'],axis=1, inplace=True)
all_times = all_times.sort_values(['data','municipio'])
display(all_times.head(3))
display(all_times[all_times.municipio == 'FORTALEZA'].head(3))

,data,municipio,macroRegiao,regiaoSaude
0,2020-02-07,ABAIARA,CARIRI,19ª REGIÃO BREJO SANTO
41,2020-02-07,ACARAPE,FORTALEZA,3ª REGIÃO MARACANAÚ
84,2020-02-07,ACARAU,SOBRAL,12ª REGIÃO ACARAÚ


,data,municipio,macroRegiao,regiaoSaude
67,2020-02-07,FORTALEZA,FORTALEZA,1ª REGIÃO FORTALEZA
242,2020-02-08,FORTALEZA,FORTALEZA,1ª REGIÃO FORTALEZA
417,2020-02-09,FORTALEZA,FORTALEZA,1ª REGIÃO FORTALEZA


In [45]:
df = pd.pivot_table(df_dia, values='quantidade', index=['municipio','data'],
                       columns=['tipo'], aggfunc=np.sum)\
                    .reset_index()\
                    .fillna(0)

all_times = all_times.merge(df, on=['municipio','data'], how='left').fillna(0)

In [46]:
cumsums = all_times.groupby(['municipio','macroRegiao','regiaoSaude', 'data']).sum().fillna(0)
cumsums = cumsums.groupby(level=0).cumsum().reset_index()
qtd_por_dia = all_times.merge(cumsums, on=['data','macroRegiao','regiaoSaude','municipio'], suffixes=['','Cum'])

In [47]:
# adicionar as informações de coordenada 
mun_coord = df_parcial.loc[:,['municipio','lat','lon']]
qtd_por_dia = qtd_por_dia.merge(mun_coord, on='municipio')

### Exportar Dados para Excel / CSV

In [48]:
# Salvando o número de casos por munício em formato CSV
df_parcial.to_csv('../data/dados_por_municipio_{}.csv'.format(date.today()))
# Salvando o número casos por dia e municipio CSV
qtd_por_dia.to_csv('../data/dados_serie_historica_{}.csv'.format(date.today()))